In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import geopandas as gpd

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [3]:
values1 = pd.read_pickle('Data/train_values_EDA.pkl')
labels = pd.read_csv('Source_Data/trainset_labels.csv')

In [4]:
values2 = values1.drop(columns=['extraction_type', 'extraction_type_class', 'water_quality', 'quantity', 'source', 'waterpoint_type_group'])

In [5]:
values2.head()

,id,amount_tsh,gps_height,longitude,latitude,basin,region_code,district_code,population,public_meeting,permit,construction_year,extraction_type_group,payment,quality_group,quantity_group,source_type,source_class,waterpoint_type
0,69572,6000.0,1390,34.938093,-9.856322,Lake Nyasa,11,5,109,1,0,1999,gravity,pay annually,good,enough,spring,groundwater,communal standpipe
1,8776,0.0,1399,34.698766,-2.147466,Lake Victoria,20,2,280,0,1,2010,gravity,never pay,good,insufficient,rainwater harvesting,surface,communal standpipe
2,34310,25.0,686,37.460664,-3.821329,Pangani,21,4,250,1,1,2009,gravity,pay per bucket,good,enough,dam,surface,communal standpipe multiple
3,67743,0.0,263,38.486161,-11.155298,Ruvuma / Southern Coast,90,63,58,1,1,1986,submersible,never pay,good,dry,borehole,groundwater,communal standpipe multiple
4,19728,0.0,0,31.130847,-1.825359,Lake Victoria,18,1,0,1,1,0,gravity,never pay,good,seasonal,rainwater harvesting,surface,communal standpipe


In [6]:
scaler = StandardScaler()
ohe = OneHotEncoder(drop='first')

In [7]:
train_num = values2.loc[:,['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population']]

In [8]:
for c in train_num.columns:
    train_num[c] = scaler.fit_transform(train_num.loc[:,c].to_numpy().reshape(-1,1))#
    #num['gps_height'] = scaler.fit_transform(values['gps_height'].to_numpy().reshape(-1,1))
#train_num['longitude'] = scaler.fit_transform(values['longitude'].to_numpy().reshape(-1,1))
#train_num['latitude'] = scaler.fit_transform(values['latitude'].to_numpy().reshape(-1,1))
#train_num['population'] = scaler.fit_transform(values['population'].to_numpy().reshape(-1,1))

In [9]:
train_num

,amount_tsh,gps_height,longitude,latitude,population
0,1.895665,1.041252,0.131052,-1.408791,-0.150399
1,-0.105970,1.054237,0.094610,1.207934,0.212290
2,-0.097630,0.025541,0.515158,0.639751,0.148660
3,-0.105970,-0.584751,0.671308,-1.849720,-0.258570
4,-0.105970,-0.964200,-0.448669,1.317271,-0.381587
...,...,...,...,...,...
59395,-0.102634,0.781553,0.470870,0.832379,-0.116463
59396,1.461977,0.784439,0.178544,-1.142092,-0.262812
59397,-0.105970,-0.964200,-0.009188,-1.033404,-0.381587
59398,-0.105970,-0.964200,0.271629,-0.228290,-0.381587


In [10]:
colt = ColumnTransformer([('basin', ohe, ['basin']), 
                          ('r_code', ohe, ['region_code']), 
                          ('d_code', ohe, ['district_code']), 
                          ('ex', ohe, ['extraction_type_group']), 
                          ('pay', ohe, ['payment']), 
                          ('qual', ohe, ['quality_group']),
                          ('quant', ohe, ['quantity_group']), 
                          ('s_type', ohe, ['source_type']),
                          ('s_class', ohe, ['source_class']),
                          ('wp_type', ohe, ['waterpoint_type'])])

In [11]:
train_cat = values2.loc[:,['basin', 'region_code', 'district_code', 'extraction_type_group', 'payment', 'quality_group', 'quantity_group', 'source_type', 'source_class', 'waterpoint_type']]
train_df = colt.fit_transform(train_cat).toarray()

In [12]:
train_cat = pd.DataFrame(train_df, columns=colt.get_feature_names())

In [13]:
train = train_num.merge(train_cat, left_index=True, right_index=True)

In [14]:
train

,amount_tsh,gps_height,longitude,latitude,population,basin__x0_Lake Nyasa,basin__x0_Lake Rukwa,basin__x0_Lake Tanganyika,basin__x0_Lake Victoria,basin__x0_Pangani,...,s_type__x0_shallow well,s_type__x0_spring,s_class__x0_surface,s_class__x0_unknown,wp_type__x0_communal standpipe,wp_type__x0_communal standpipe multiple,wp_type__x0_dam,wp_type__x0_hand pump,wp_type__x0_improved spring,wp_type__x0_other
0,1.895665,1.041252,0.131052,-1.408791,-0.150399,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-0.105970,1.054237,0.094610,1.207934,0.212290,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.097630,0.025541,0.515158,0.639751,0.148660,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-0.105970,-0.584751,0.671308,-1.849720,-0.258570,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-0.105970,-0.964200,-0.448669,1.317271,-0.381587,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,-0.102634,0.781553,0.470870,0.832379,-0.116463,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
59396,1.461977,0.784439,0.178544,-1.142092,-0.262812,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
59397,-0.105970,-0.964200,-0.009188,-1.033404,-0.381587,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
59398,-0.105970,-0.964200,0.271629,-0.228290,-0.381587,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [15]:
train_X, test_X, train_y, test_y = train_test_split(train, labels['status_group'], test_size=0.25, random_state=42)

In [16]:
rf = RandomForestClassifier()
params = {'n_estimators':[10, 100, 200, 500], 'criterion':['gini', 'entropy']}
rs = RandomizedSearchCV(rf, params, n_iter=5)

In [17]:
rf.fit(train_X, train_y)
rf.score(test_X, test_y)

0.7896969696969697

In [18]:
rs.fit(train_X, train_y)
rs.best_estimator_
rs.cv_results_

{'mean_fit_time': array([ 1.07259064,  1.24959879, 48.35621433, 22.92786884,  9.74193082]),
 'std_fit_time': array([0.01359998, 0.02885357, 0.21185919, 0.08532602, 0.0372907 ]),
 'mean_score_time': array([0.05199747, 0.05079904, 1.77680006, 0.73236728, 0.3628211 ]),
 'std_score_time': array([0.00123864, 0.00116518, 0.01773562, 0.06894357, 0.00547369]),
 'param_n_estimators': masked_array(data=[10, 10, 500, 200, 100],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_criterion': masked_array(data=['gini', 'entropy', 'gini', 'entropy', 'gini'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 10, 'criterion': 'gini'},
  {'n_estimators': 10, 'criterion': 'entropy'},
  {'n_estimators': 500, 'criterion': 'gini'},
  {'n_estimators': 200, 'criterion': 'entropy'},
  {'n_estimators': 100, 'criterion': 'gini'}],
 'split0_test_score': array([0.7

In [19]:
rf.classes_

array(['functional', 'functional needs repair', 'non functional'],
      dtype=object)

In [20]:
features = pd.Series(rf.feature_importances_, index=train.columns)

In [21]:
features.sort_values(ascending=False).head(20)

longitude                                  0.180459
latitude                                   0.178685
gps_height                                 0.091914
population                                 0.062645
quant__x0_enough                           0.046702
amount_tsh                                 0.032726
ex__x0_other                               0.028831
quant__x0_insufficient                     0.026684
wp_type__x0_other                          0.025346
wp_type__x0_communal standpipe             0.013630
wp_type__x0_hand pump                      0.013149
quant__x0_seasonal                         0.011532
pay__x0_pay per bucket                     0.011063
wp_type__x0_communal standpipe multiple    0.009598
pay__x0_unknown                            0.009383
d_code__x0_1                               0.009298
ex__x0_gravity                             0.009101
ex__x0_nira/tanira                         0.009008
qual__x0_good                              0.008747
pay__x0_pay 

In [22]:
submit_X = pd.read_csv('Source_data/testset_values.csv')

In [23]:
sub_num = submit_X.loc[:,['amount_tsh', 'gps_height', 'longitude', 'latitude', 'population']]
for c in sub_num.columns:
    sub_num[c] = scaler.fit_transform(sub_num.loc[:,c].to_numpy().reshape(-1,1))
sub_num

,amount_tsh,gps_height,longitude,latitude,population
0,-0.128571,1.939784,0.186445,0.552598,0.291567
1,-0.128571,1.322052,0.393626,0.807803,0.246837
2,-0.128571,1.319159,0.107125,0.231366,0.672837
3,-0.128571,-0.561525,0.606182,-1.269746,0.140337
4,0.070562,0.875028,0.143265,-1.790621,-0.264363
...,...,...,...,...,...
14845,-0.128571,-0.898602,0.726710,-0.305409,-0.349563
14846,0.269695,-0.947789,0.514201,0.113679,5.912637
14847,-0.128571,1.187511,0.102870,0.373767,0.033837
14848,-0.128571,0.495998,0.207973,-1.666075,-0.072663


In [24]:
sub_cat = submit_X.loc[:,['basin', 'region_code', 'district_code', 'extraction_type_group', 'payment', 'quality_group', 'quantity_group', 'source_type', 'source_class', 'waterpoint_type']]
sub_df = colt.transform(sub_cat).toarray()

In [25]:
sub_cat = pd.DataFrame(sub_df, columns=colt.get_feature_names())

In [26]:
sub_train = sub_num.merge(sub_cat, left_index=True, right_index=True)

In [27]:
sub_train

,amount_tsh,gps_height,longitude,latitude,population,basin__x0_Lake Nyasa,basin__x0_Lake Rukwa,basin__x0_Lake Tanganyika,basin__x0_Lake Victoria,basin__x0_Pangani,...,s_type__x0_shallow well,s_type__x0_spring,s_class__x0_surface,s_class__x0_unknown,wp_type__x0_communal standpipe,wp_type__x0_communal standpipe multiple,wp_type__x0_dam,wp_type__x0_hand pump,wp_type__x0_improved spring,wp_type__x0_other
0,-0.128571,1.939784,0.186445,0.552598,0.291567,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.128571,1.322052,0.393626,0.807803,0.246837,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.128571,1.319159,0.107125,0.231366,0.672837,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.128571,-0.561525,0.606182,-1.269746,0.140337,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.070562,0.875028,0.143265,-1.790621,-0.264363,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14845,-0.128571,-0.898602,0.726710,-0.305409,-0.349563,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
14846,0.269695,-0.947789,0.514201,0.113679,5.912637,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
14847,-0.128571,1.187511,0.102870,0.373767,0.033837,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
14848,-0.128571,0.495998,0.207973,-1.666075,-0.072663,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [28]:
submition = rf.predict(sub_train)

In [29]:
submission1 = pd.DataFrame(submition, columns=['status_group'])
submission1['id'] = submit_X['id']

In [30]:
submission1.to_csv('submission1_BW.csv')